In [1]:
%cd ..

/home/oleg/asr-eval


In [3]:
# type: ignore

from typing import Any

from jiwer import wer
from datasets import Dataset, load_dataset, Audio
import numpy as np
import pandas as pd

from asr_eval.streaming.sender import StreamingAudioSender
from asr_eval.streaming.caller import transribe_parallel
from asr_eval.streaming.models.vosk import VoskStreaming
from asr_eval.streaming.model import PartialTranscription

In [4]:
# type: ignore

dataset: Dataset = load_dataset('bond005/podlodka_speech')['test'].take(10)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))
df = pd.DataFrame(list(dataset))

def get_id(audio: dict[str, Any]) -> str:
    return audio['path']

def get_streaming_sender(id: str, audio: dict[str, Any]) -> dict[str, Any]:
    assert audio['sampling_rate'] == 16_000
    return StreamingAudioSender(
        audio=np.int16(audio['array'] * 32768).tobytes(),
        id=id,
        sampling_rate=16_000,
        real_time_interval_sec=1 / 25,
        speed_multiplier=25,
        track_history=True,
    )

df['id'] = df.audio.apply(get_id)
df['sender'] = df.apply(lambda row: get_streaming_sender(row.id, row.audio), axis=1)

In [5]:
# type: ignore

asr = VoskStreaming(model_name='vosk-model-ru-0.42')
asr.start_thread()

results = transribe_parallel(asr=asr, senders=df.sender, n_threads=8)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/oleg/.cache/vosk/vosk-model-ru-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /home/oleg/.cache/vosk/vosk-model-ru-0.42/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:297) Loading words from /home/oleg/.cache/vosk/vosk-model-ru-0.42/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:30

Transcribing sound_test_0001.wav
Transcribing sound_test_0002.wav
Transcribing sound_test_0003.wav
Transcribing sound_test_0004.wav
Transcribing sound_test_0005.wav
Transcribing sound_test_0006.wav
Transcribing sound_test_0007.wav
Transcribing sound_test_0008.wav
Transcribed sound_test_0001.wav: и поэтому использовать их в повседневности не получается мы вынуждены отступать
Transcribing sound_test_0009.wav
Transcribed sound_test_0007.wav: неужто не может быть какое-то количество дискретных столбиков где каждый [...]
Transcribing sound_test_0010.wav
Transcribed sound_test_0002.wav: максимально ухудшить идеальную систему в воде туда какие-то элементы или [...]
Transcribed sound_test_0006.wav: и мне кажется абсолютно все замечали что детские крики раздражают там и ты [...]
Transcribed sound_test_0004.wav: да это отсутствие долго живущие бранчей другими словами отсутствие какому- [...]
Transcribed sound_test_0003.wav: человек недавно ставший тимлидом и который сказал знаешь заходит в кабин

In [6]:
df['preds'] = [PartialTranscription.join(x) for x in results.values()]

In [7]:
# type: ignore

wer = wer(reference=df['transcription'].tolist(), hypothesis=df['preds'].tolist())
print(wer)

0.5010845986984815


In [8]:
for sender in df.sender:
    sender.remove_waveforms_from_history()

In [9]:
df.sender.iloc[0].history

[InputChunk(data=b'', start_time=0.0, end_time=1.0, put_timestamp=1747820900.9349077, get_timestamp=1747820900.9356358),
 InputChunk(data=b'', start_time=1.0, end_time=2.0, put_timestamp=1747820900.9751303, get_timestamp=1747820901.0716043),
 InputChunk(data=b'', start_time=2.0, end_time=3.0, put_timestamp=1747820901.015362, get_timestamp=1747820901.0751026),
 InputChunk(data=b'', start_time=3.0, end_time=4.0, put_timestamp=1747820901.0555985, get_timestamp=1747820901.0786216),
 InputChunk(data=b'', start_time=4.0, end_time=5.0, put_timestamp=1747820901.0957313, get_timestamp=1747820901.9957561),
 InputChunk(data=b'', start_time=5.0, end_time=6.0, put_timestamp=1747820901.1359332, get_timestamp=1747820902.4124331),
 InputChunk(data=b'', start_time=6.0, end_time=7.0, put_timestamp=1747820901.176144, get_timestamp=1747820902.847347),
 InputChunk(data=b'', start_time=7.0, end_time=8.0, put_timestamp=1747820901.2163422, get_timestamp=1747820903.3151884),
 InputChunk(data=b'', start_time=8.

In [10]:
results['sound_test_0001.wav']

[OutputChunk(data=PartialTranscription(id='__latest__', text='', final=False), put_timestamp=1747820900.960514, get_timestamp=1747820900.9606042),
 OutputChunk(data=PartialTranscription(id='__latest__', text='', final=False), put_timestamp=1747820901.0720465, get_timestamp=1747820901.0724173),
 OutputChunk(data=PartialTranscription(id='__latest__', text='', final=False), put_timestamp=1747820901.0755286, get_timestamp=1747820901.0755775),
 OutputChunk(data=PartialTranscription(id='__latest__', text='', final=False), put_timestamp=1747820901.190621, get_timestamp=1747820901.190871),
 OutputChunk(data=PartialTranscription(id='__latest__', text='и поэтому', final=False), put_timestamp=1747820902.05459, get_timestamp=1747820902.0547245),
 OutputChunk(data=PartialTranscription(id='__latest__', text='и поэтому', final=False), put_timestamp=1747820902.4655032, get_timestamp=1747820902.465892),
 OutputChunk(data=PartialTranscription(id='__latest__', text='и поэтому использовать их в', final=Fa